In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import onnxruntime

import onnx

class MyModel(nn.Module):

    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
torch_model = MyModel()

torch_input = torch.randn(1, 1, 32, 32)
onnx_program = torch.onnx.dynamo_export(torch_model, torch_input)

/home/oksana/lab/dl/lib/python3.10/site-packages/torch/onnx/_internal/exporter.py:136: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


In [8]:
onnx_program.save("my_image_classifier.onnx")

In [9]:
onnx_model = onnx.load("my_image_classifier.onnx")
onnx.checker.check_model(onnx_model)

In [22]:
onnx_input = onnx_program.adapt_torch_inputs_to_onnx(torch_input)
print(f"Input length: {len(onnx_input)}")
print(f"Sample input: {onnx_input}")

ort_session = onnxruntime.InferenceSession("./my_image_classifier.onnx", providers=['CPUExecutionProvider'])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)}

onnxruntime_outputs = ort_session.run(None, onnxruntime_input)

Input length: 1
Sample input: (tensor([[[[ 1.0059, -0.7643, -0.4352,  ...,  0.9810,  0.5191, -1.4382],
          [ 0.0483,  0.2972,  0.4813,  ...,  0.0178,  0.7878, -0.3140],
          [ 0.1914,  0.2130, -0.3057,  ...,  0.9278,  0.7526, -0.9128],
          ...,
          [ 0.2247, -2.0818, -2.6542,  ...,  0.5326,  2.6555, -1.0239],
          [ 1.0937,  1.8356, -0.4176,  ...,  0.1687, -0.0972,  1.0805],
          [ 1.3127, -0.8750,  1.0790,  ..., -0.9243,  0.2615,  0.9691]]]]),)


In [23]:
onnxruntime_input

{'l_x_': array([[[[ 1.0058511 , -0.76430833, -0.43518716, ...,  0.98102295,
            0.51906806, -1.438247  ],
          [ 0.04827968,  0.29716635,  0.48133898, ...,  0.01778525,
            0.78782237, -0.31397158],
          [ 0.19143265,  0.2129756 , -0.3056532 , ...,  0.92784697,
            0.7526169 , -0.9128105 ],
          ...,
          [ 0.2247328 , -2.0818071 , -2.6542196 , ...,  0.53263646,
            2.65547   , -1.0239217 ],
          [ 1.0937264 ,  1.8355749 , -0.41756773, ...,  0.16866119,
           -0.09717131,  1.0805334 ],
          [ 1.3127235 , -0.87498254,  1.0790318 , ..., -0.92430913,
            0.26153386,  0.96910375]]]], dtype=float32)}

In [92]:
torch_outputs = torch_model(torch_input)
torch_outputs = onnx_program.adapt_torch_outputs_to_onnx(torch_outputs)

In [94]:
assert len(torch_outputs) == len(onnxruntime_outputs)
for torch_output, onnxruntime_output in zip(torch_outputs, onnxruntime_outputs):
    torch.testing.assert_close(torch_output, torch.tensor(onnxruntime_output))

print("PyTorch and ONNX Runtime output matched!")
print(f"Output length: {len(onnxruntime_outputs)}")
print(f"Sample output: {onnxruntime_outputs}")

PyTorch and ONNX Runtime output matched!
Output length: 1
Sample output: [array([[ 0.01185731, -0.05390121, -0.00806683,  0.07318756, -0.08826195,
        -0.14342554,  0.09486622,  0.03274141,  0.04959921,  0.03379258]],
      dtype=float32)]
